In [1]:
import pandas as pd
df= pd.read_csv(r"E:\Aegis\Project\Data_set\rechatterbotpdf/ED_1.1.csv")

df.size
#print df["QA"][391]
import re
#def cleaning_text(review):
    #cleaned =re.sub("[^a-zA-Z/:.?]", " ",review)
    #words =cleaned.lower().split()
    #return(" ".join( words ))
new_words =[]
for i in range(0,df.size -1, 2):
    new_words.append(df["QA"][i])

new_words
df1 = pd.DataFrame({'Questions': new_words})

In [2]:
#print df1
#df1 = df1.drop_duplicates()
df1.to_csv("chat_sentiment.csv", index =False)

In [3]:
import pandas as pd
df1 = pd.read_csv(r"E:\Aegis\Project\Data_set\rechatterbotpdf/chat_sentiment.csv")

from bs4 import BeautifulSoup  
import nltk
import re
from nltk.corpus import stopwords


In [4]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words )) 

In [5]:
print df1.columns

Index([u'Questions', u'Sentiment'], dtype='object')


In [6]:
clean_review = review_to_words( df1["Questions"][0] )
print clean_review

business analytics


C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file C:\Users\Ajith Vajrala\Anaconda2\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [7]:
num_reviews = df1["Questions"].size
clean_train_reviews = []

for i in xrange( 0, num_reviews ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append( review_to_words( df1["Questions"][i] ) )

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

 
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000) 

train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

In [9]:
vocab = vectorizer.get_feature_names()
print vocab

[u'able', u'accepted', u'access', u'admission', u'aegis', u'analytics', u'application', u'apply', u'asked', u'attend', u'available', u'babd', u'better', u'bi', u'big', u'business', u'campus', u'career', u'center', u'certification', u'change', u'class', u'coding', u'companies', u'course', u'criteria', u'curriculum', u'data', u'dates', u'deal', u'delivery', u'depend', u'different', u'done', u'education', u'eligibile', u'eligibility', u'end', u'epgp', u'ex', u'executive', u'expected', u'experience', u'faculty', u'feedback', u'field', u'find', u'free', u'full', u'future', u'get', u'good', u'graduate', u'guarantee', u'guaranteed', u'guarnatee', u'hybrid', u'ibm', u'intelligence', u'intenship', u'internship', u'interview', u'involved', u'job', u'jobs', u'join', u'kind', u'know', u'loan', u'location', u'mba', u'means', u'members', u'middle', u'miss', u'missed', u'mode', u'model', u'models', u'module', u'ms', u'much', u'necessary', u'need', u'needed', u'old', u'online', u'opinion', u'opportuni

In [10]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, df1["Sentiment"] )

Training the random forest...


In [11]:
clean_test_reviews = [] 
clean_review = review_to_words("what is eligibility criteria" )
clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)
print result

['Interested']
